In [49]:
import pandas as pd
import numpy as np

In [50]:
data = pd.read_csv("train_v9rqX0R.csv")

In [51]:
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [52]:
#understanding categorical col

print(data["Item_Fat_Content"].unique())
print(data["Item_Type"].unique())
print(data["Outlet_Size"].unique())
print(data["Outlet_Location_Type"].unique())
print(data["Outlet_Type"].unique())

['Low Fat' 'Regular' 'low fat' 'LF' 'reg']
['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']
['Medium' nan 'High' 'Small']
['Tier 1' 'Tier 3' 'Tier 2']
['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']


In [53]:
#handling categorical col
replace_map={
    'low fat' : 'Low fat',
    'lf': 'Low fat',
    'reg':'Regular'
}
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace(replace_map)

cat_col = data.select_dtypes(include=['object','category']).columns
for col in cat_col:
    mode_value = data[col].mode()[0]
    data[col] = data[col].fillna(mode_value)
   
print(data['Item_Fat_Content'].unique())
print(data['Outlet_Size'].unique())

['Low Fat' 'Regular' 'Low fat' 'LF']
['Medium' 'High' 'Small']


In [54]:
#understanding numerical col
num_cols = data.select_dtypes(include=["number"]).columns
print(data[num_cols].isna().sum())

Item_Weight                  1463
Item_Visibility                 0
Item_MRP                        0
Outlet_Establishment_Year       0
Item_Outlet_Sales               0
dtype: int64


In [55]:
#handling numerical col
data['Item_Weight'] = data['Item_Weight'].fillna(data['Item_Weight'].mean())
print(data['Item_Weight'].isna().sum())

0


In [64]:
#train test split
from sklearn.model_selection import train_test_split

X = data.drop(columns=['Item_Outlet_Sales'])
Y = data['Item_Outlet_Sales']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)


In [65]:
#applying one hot encoding
#data_ohe = pd.get_dummies(data[cat_col])
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

cat_col = X_train.select_dtypes(include=['object','category']).columns
num_cols = X_train.select_dtypes(include=["number"]).columns

preprocessor = ColumnTransformer(
    transformers=[
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore',sparse_output=False), cat_col)
])


In [66]:
# model fitting
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr = Pipeline(steps=[
    ('preprocessor' ,preprocessor),
    ('regressor' ,LinearRegression())
])
lr.fit(X_train, Y_train)
Y_pred = lr.predict(X_test)


In [67]:
#check performance -> rmse
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
print(rmse)

1273.6697103421452
